In [1]:
# from sklearnex import patch_sklearn
# patch_sklearn()  # Activate Intel optimizations for scikit-learn
from sklearn.svm import SVC
from sklearn.utils import shuffle
from sklearn.metrics import classification_report
import numpy as np
from tqdm import tqdm
import pandas as pd

In [2]:
# --- CONFIG --- #
train_files = [
    "parsed_emg/subject_1/trial_11(right).csv",
    # "parsed_emg/subject_1/trial_3(right).csv",
    # "parsed_emg/subject_1/trial_5(right).csv",
    # "parsed_emg/subject_1/trial_7(right).csv",
    # "parsed_emg/subject_1/trial_9(right).csv",
]

test_file = "parsed_emg/subject_1/trial_1(right).csv"
label_col = 5

# --- Data Loader --- #
def load_csv(path):
    df = pd.read_csv(path, skiprows=1)
    df = df.drop(columns=df.columns[0], axis=1)
    X = df.iloc[:, 1].values.astype("float64").reshape(-1, 1)  # Make it 2D
    y = df.iloc[:, label_col].values.astype("int32")
    return X, y

In [ ]:
# --- Data Loading (unchanged) --- #
X_train_list, y_train_list = [], []
for path in tqdm(train_files, desc="Loading training data"):
    X, y = load_csv(path)
    X_train_list.append(X)
    y_train_list.append(y)

X_train = np.vstack(X_train_list)
y_train = np.hstack(y_train_list)


print("shuffling data")
# Shuffle training data
X_train, y_train = shuffle(X_train, y_train, random_state=42)

# --- Train SVM (Intel-optimized) --- #
print("Training SVM with Intel acceleration...")
model = SVC(
    kernel='rbf',
    decision_function_shape='ovr',
    verbose=True,
    # Optional: Set cache_size to use more RAM (faster for large datasets)
    cache_size=8000  # MB, adjust based on your system RAM
)
model.fit(X_train, y_train)
print("Training complete.")

# --- Test --- #
X_test, y_test = load_csv(test_file)
y_pred = model.predict(X_test)

# --- Results --- #
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

Loading training data: 100%|██████████| 1/1 [00:00<00:00,  3.84it/s]


shuffling data
Training SVM with Intel acceleration...
[LibSVM]